# AeroBot - Modeling phase - Iteration 1


Following the initial data audit and visualization phase of our project 'AeroBot', we now focus on NLP of the Narratives and try our first predictive algorithms based on the label 'Primary Problem'.

## About

First NLP models of our project.


**Tutor**

* Alban THUET

**Authors:**

* Hélène ASSIR
* Hichem HADJI  
* [Ioannis STASINOPOULOS](https://www.linkedin.com/in/ioannis-stasinopoulos/)

</br>

---
</br>

**Version History**

Version | Date       | Author | Modification
--------|----------- | ------ | --------------------------
X.X     | XX/XX/2022 | A.B    | modif
1.0     | 11/06/2022 | I.S    | Document creation

## Load the data

### Mount & explore your Google Drive

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
#check your present working directory 
%pwd

'/content/drive/My Drive/Data_Science/Formations/DataScienceTest/projet/AeroBot'

In [ ]:
# move to the desired location (adapt to your folder-tree-structure):
%cd /content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/

/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot


In [ ]:
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

ASRS_20y_data.csv			   Narratives.pkl
D1_DRAFT_Dataviz_Ioannis_20220527.gslides  plane.png


### Copy Ioannis' file to your Google Drive

This section shows you how to copy the file from Ioannis's Google Drive into the pwd of your Google Drive.

**Uncomment and execute the cells, if desired.**

Source: 
https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

In [ ]:
# libraries for the files in google drive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# file_id = '16My7pG4Ral93VIAQrCTpcMn5JTa79DGo' 
# # This is the id of the ASRS_20y_data.csv file in Ioannis' Google Drive, which he has shared with you
# # (To find the id of a file, click 'share' and 'get link' and copy the large nonsense part of the link

# # 'download' the file to your present working directory 
# download = drive.CreateFile({'id': file_id})
# download.GetContentFile('copied_file.csv')
# df  = pd.read_csv("copied_file.csv")
# df.head()

## Import packages

In [ ]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns

import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import itertools # Pour créer des iterateurs

######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# Define global plot parameters for better readability and consistency among plots
# A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 23
plt.rcParams['xtick.labelsize'] = 23
plt.rcParams['ytick.labelsize'] = 23
plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus

###############################
# ML preprocessing and models
###############################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report, confusion_matrix

import pickle as pkl # Saving data externally

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df = pd.read_csv('ASRS_20y_data.csv', low_memory=False)
# See https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options

In [ ]:
df.head(3)

,Unnamed: 0,ACN,Date,Local Time Of Day,Locale Reference,State Reference,Relative Position.Angle.Radial,Relative Position.Distance.Nautical Miles,Altitude.AGL.Single Value,Altitude.MSL.Single Value,Latitude / Longitude (UAS),Flight Conditions,Weather Elements / Visibility,Work Environment Factor,Light,Ceiling,RVR.Single Value,ATC / Advisory,Aircraft Operator,Make Model Name,Aircraft Zone,Crew Size,Operating Under FAR Part,Flight Plan,Mission,Nav In Use,Flight Phase,Route In Use,Airspace,Maintenance Status.Maintenance Deferred,Maintenance Status.Records Complete,Maintenance Status.Released For Service,Maintenance Status.Required / Correct Doc On Board,Maintenance Status.Maintenance Type,Maintenance Status.Maintenance Items Involved,Cabin Lighting,Number Of Seats.Number,Passengers On Board.Number,Crew Size Flight Attendant.Number Of Crew,Airspace Authorization Provider (UAS),Operating Under Waivers / Exemptions / Authorizations (UAS),Waivers / Exemptions / Authorizations (UAS),Airworthiness Certification (UAS),Weight Category (UAS),Configuration (UAS),Flight Operated As (UAS),Flight Operated with Visual Observer (UAS),Control Mode (UAS),Flying In / Near / Over (UAS),Passenger Capable (UAS),Type (UAS),Number of UAS Being Controlled (UAS),Aircraft Component,Manufacturer,Aircraft Reference,Problem,ATC / Advisory.1,Aircraft Operator.1,Make Model Name.1,Aircraft Zone.1,Crew Size.1,Operating Under FAR Part.1,Flight Plan.1,Mission.1,Nav In Use.1,Flight Phase.1,Route In Use.1,Airspace.1,Maintenance Status.Maintenance Deferred.1,Maintenance Status.Records Complete.1,Maintenance Status.Released For Service.1,Maintenance Status.Required / Correct Doc On Board.1,Maintenance Status.Maintenance Type.1,Maintenance Status.Maintenance Items Involved.1,Cabin Lighting.1,Number Of Seats.Number.1,Passengers On Board.Number.1,Crew Size Flight Attendant.Number Of Crew.1,Airspace Authorization Provider (UAS).1,Operating Under Waivers / Exemptions / Authorizations (UAS).1,Waivers / Exemptions / Authorizations (UAS).1,Airworthiness Certification (UAS).1,Weight Category (UAS).1,Configuration (UAS).1,Flight Operated As (UAS).1,Flight Operated with Visual Observer (UAS).1,Control Mode (UAS).1,Flying In / Near / Over (UAS).1,Passenger Capable (UAS).1,Type (UAS).1,Number of UAS Being Controlled (UAS).1,Location Of Person,Location In Aircraft,Reporter Organization,Function,Qualification,Experience,Cabin Activity,Human Factors,Communication Breakdown,UAS Communication Breakdown,ASRS Report Number.Accession Number,Location Of Person.1,Location In Aircraft.1,Reporter Organization.1,Function.1,Qualification.1,Experience.1,Cabin Activity.1,Human Factors.1,Communication Breakdown.1,UAS Communication Breakdown.1,ASRS Report Number.Accession Number.1,Anomaly,Miss Distance,Were Passengers Involved In Event,Detector,When Detected,Result,Contributing Factors / Situations,Primary Problem,Narrative,Callback,Narrative.1,Callback.1,Synopsis,Unnamed: 125
0,0,986560,201201,1801-2400,A11.TRACON,AK,NaN,NaN,NaN,8000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRACON A11,Air Taxi,Super King Air 200,NaN,1.0,Part 135,IFR,Passenger,NaN,Descent,Vectors,Class E ANC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Facility A11.TRACON,NaN,Government,Departure; Approach,Air Traffic Control Developmental,NaN,NaN,Situational Awareness,NaN,NaN,986560.0,NaN,Flight Deck,Air Taxi,Captain; Pilot Flying,NaN,Flight Crew Last 90 Days 46; Flight Crew Total...,NaN,NaN,NaN,NaN,986993.0,Deviation - Altitude Excursion From Assigned A...,NaN,NaN,Person Air Traffic Control,NaN,Air Traffic Control Issued New Clearance,Human Factors,Human Factors,A BE20 was inbound to ANC via YESKA. He was to...,NaN,I was descending to 8;000 FT proceeding direct...,NaN,A11 Controller described a descent below MVA; ...,NaN
1,1,988077,201201,000

In [ ]:
print("The raw dataset has a total of", len(df), "entries.",
     "\n/!\ These include unmanned-aircraft-related entries; yet the latter are insignificantly low in number (see below)")

The raw dataset has a total of 108407 entries. 
/!\ These include unmanned-aircraft-related entries; yet the latter are insignificantly low in number (see below)


## Remove UAS-related data

A UAS (Unmanned Aircraft Systems) includes not only the UAV (or drone), but also the person on the ground controlling the flight and the system in place that connects both of them.

### Find out how many UAS-entries we have

In [ ]:
features_UAS_all = [
###################################
### FEATURES RELATED TO DRONES  ###
###################################
 'Latitude / Longitude (UAS)',
    'Operating Under Waivers / Exemptions / Authorizations (UAS)',
  'Waivers / Exemptions / Authorizations (UAS)',
  'Airworthiness Certification (UAS)', # Only 'Standard' value
  'Weight Category (UAS)',
  'Configuration (UAS)',
  'Flight Operated As (UAS)',
  'Flight Operated with Visual Observer (UAS)', # BVLOS (Beyond Visual Line of Sight) is a term relating to the operation of UAVs (unmanned aerial vehicles) and drones at distances outside the normal visible range of the pilot.
 'Control Mode (UAS)',
 'Flying In / Near / Over (UAS)',
 'Passenger Capable (UAS)', # There are only NaNs and 'N'
 'Type (UAS)',
 'Number of UAS Being Controlled (UAS)',
 'UAS Communication Breakdown',    
 'Airspace Authorization Provider (UAS)', # only NaN's in this feature --> drop ?

    
##################################################
### Additional entries for the above variables ###
##################################################
 'Airspace Authorization Provider (UAS).1',
 'Operating Under Waivers / Exemptions / Authorizations (UAS).1',
 'Waivers / Exemptions / Authorizations (UAS).1',
 'Airworthiness Certification (UAS).1',
 'Weight Category (UAS).1',
 'Configuration (UAS).1',
 'Flight Operated As (UAS).1',
 'Flight Operated with Visual Observer (UAS).1',
 'Control Mode (UAS).1',
 'Flying In / Near / Over (UAS).1',
 'Passenger Capable (UAS).1',
 'Type (UAS).1',
 'Number of UAS Being Controlled (UAS).1',    
##################################################
]

In [ ]:
# Search and gather the rows that contain non-nan values for any of the UAS features into dataframes

files = [] # instantiate empty list

for feat in features_UAS_all:
    
    # append pandas DataFrames to 'files' --> files becomes list of df's.
    files.append(df.loc[df[feat].isna() == False])

# concatenate df's into 1 df:
df_UAS = pd.concat(files)

# Drop the duplicates; there definitely exist many, because we searched column-by-column
df_UAS = df_UAS.drop_duplicates(keep= 'first')

In [ ]:
print("The data contain", len(df_UAS), "UAS-related entries.")

The data contain 166 UAS-related entries.


This quantity of UAS entries seems reasonable, because these data have NaN's in many MANNED-aircraft-related features, as shown in the next line. Note that 'Make Model Name' has no NaNs.

In [ ]:
df_UAS.isna().sum()

Unnamed: 0                                                         0
ACN                                                                0
Date                                                               0
Local Time Of Day                                                 18
Locale Reference                                                   1
State Reference                                                    0
Relative Position.Angle.Radial                                   138
Relative Position.Distance.Nautical Miles                         79
Altitude.AGL.Single Value                                         83
Altitude.MSL.Single Value                                         98
Latitude / Longitude (UAS)                                       166
Flight Conditions                                                 20
Weather Elements / Visibility                                     68
Work Environment Factor                                          162
Light                             

### Drop UAS-related features and data

In [ ]:
# remove rows
df = df.drop(df_UAS.index, axis = 0)

In [ ]:
# Check: 
len(df) + len(df_UAS)

108407

In [ ]:
# remove UAS-columns
df = df.drop(columns= features_UAS_all)

In [ ]:
print('df length after UAS entry removal:', len(df))

df length after UAS entry removal: 108241


## TESTS FOR DEBUGGING

In [ ]:
# Familiarize with the str.find() function 
print('a dog is not dog'.find('dog'))
print('a dog is not dog'.find('cat'))

2
-1


In [ ]:
counts = 0
for i in range(len(df)):
  if str(df['Anomaly'].iloc[i]).find('Deviation / Discrepancy - Procedural') != -1:
    counts += 1
print(counts)

64680


In [ ]:
anomaly_uniques = str(df['Anomaly'].unique())

#anomaly_all = str(df['Anomaly'])
# ALTERNATIVE:
# Write all the entries of 'Anomaly' into a single string
anomaly_all = ''
anomaly_all = ''.join(str(entry) for entry in df['Anomaly']) # do not include a ' ' before / after the '$' !

In [ ]:
# Compile Regular Expressions

# RegEx for 'Deviation / Discrepancy - Procedural' and its sublabels:
r6 = re.compile(r"""
                  Deviation\s/\sDiscrepancy\s\-\sProcedural       # use \s instead of ' ', because verbose ignores the latter! 
                  
                  [a-zA-Z0-9_                  # define custom list of characters that should be found at least once 
                                               # corresponds to \w and we extend it with more characters below
                  
                  \-                           # hyphen
                  
                  \s                           # whitespace
                  
                  \/                           # forward slash
                  
                  \(\)]                        # left or right bracket; if detected it means you have not dropped all UAS entries
                                               # keep the ']' at the end; otherwise less matches are found
                   
                  +                            # '+' means 'find at least 1 occurence'
                                
               """, 
               re.VERBOSE)                     # VERBOSE allows us to write the RegEx in the above, expanded way --> more readable

In [ ]:
# Simpler variant of this RegEx:

# RegEx for 'Deviation / Discrepancy - Procedural' and its sublabels:
r6_1 = re.compile(r"""
                  Deviation\s/\sDiscrepancy\s\-\sProcedural+
               """, 
               re.VERBOSE)                     # VERBOSE allows us to write the RegEx in the above, expanded way --> more readable

In [ ]:
findings_in_unique = r6_1.findall(anomaly_uniques)
len(findings_in_unique)

10

In [ ]:
# This finds the RegEx even if it appears multiple time in a single data entry
# That's why it yields A LOT of results
findings_in_all = r6_1.findall(anomaly_all)
len(findings_in_all)

104323

In [ ]:
# Use the RegEx line-by-line in our dataframe
counts = 0
for i in range(len(df)):

  if len(r6.findall(str(df['Anomaly'].iloc[i]))) > 0:
    counts += 1
print(counts)

64680


In [ ]:
# Identical result for the r6_1 variant:
counts = 0
for i in range(len(df)):

  if len(r6_1.findall(str(df['Anomaly'].iloc[i]))) > 0:
    counts += 1
print(counts)

64680
